In [1]:
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests
import os
import pandas as pd
from selenium import webdriver
from time import sleep
import random


In [2]:
# URL, chromedriver path, minimum request & sleep interval
url = input("Enter website url: ")

output = input("Enter output file name: ")
minreq = 5000
timer = random.randint(2, 5)

Enter website url: https://www.mycareersfuture.sg/search?search=research%20scientist&sortBy=new_posting_date&page=0
Enter output file name: Research_Scientist_MCF


In [3]:
# Execution of URL in WebDriver and retrieve source
chromedriver = "chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(executable_path=chromedriver)
driver.get(url.format(0))
print("\n","Scraping now!: {} ...wait for output file".format(url),"\n")
sleep(timer)
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')


 Scraping now!: https://www.mycareersfuture.sg/search?search=research%20scientist&sortBy=new_posting_date&page=0 ...wait for output file 



In [4]:
# Closes cookies warning - at the top
button = driver.find_element_by_xpath('//*[@id="dismiss-button"]')
button.click()
sleep(1)

# Closes "Add Skills" pop-up - at the bottom
button = driver.find_element_by_xpath('//*[@class="tr pointer OverlayNavigation__icon-cross___1wfSE"]')
button.click()
sleep(1)
soup = BeautifulSoup(driver.page_source, 'lxml')

In [ ]:
# Initialized all variables with empty lists
job_id = []
job_title = []
company_name = []
job_location = []
job_level = []
min_salary = []
max_salary = []
salary_type = []
days_posted = []
job_desc = []
job_req = []
job_type = []
job_industry = []
job_url = []
job_count = 0
jip_count = 0
is_responsive = []
is_contract = []
is_intern = []
is_temp = []
is_perm = []

In [ ]:
while job_count < minreq:
    
    list_of_jobs = soup.findAll("p", {"name":"company"})
    if jip_count < len(list_of_jobs):
        
        # Job ID
        job_id_var = soup.findAll("div", "absolute top-0 right-0 JobCard__bookmark-box___pdZMT")
        for x in range(len(job_id_var)):
            job_id.append(job_id_var[x].div.input["id"])
            
        # Job Title
        job_title_var = soup.findAll("h1", {"name":"job_title"})
        for x in range(len(job_title_var)):
            job_title.append(job_title_var[x].text)
            
        # Company Name
        company_name_var = soup.findAll("p", {"name":"company"})
        for x in range(len(company_name_var)):
            company_name.append(company_name_var[x].text)            
       
        # Job Links
        job_link_var = soup.findAll("a", "bg-white mb3 w-100 dib v-top pa3 no-underline flex-ns flex-wrap JobCard__card___22xP3")
        
        for x in range(len(job_link_var)):
            job_url.append("https://www.mycareersfuture.sg"+job_link_var[x]["href"])            
            
            # Next Level Stuff
            button = driver.find_element_by_xpath('//*[@href="'+job_link_var[x]["href"]+'"]')
            button.click()
            sleep(timer)
            soup_level2 = BeautifulSoup(driver.page_source, 'lxml')
            
            # Location
            job_location_var = soup_level2.findAll("p", {"id":"address"})
            if len(job_location_var) > 0:
                job_location.append(job_location_var[0].text)
            else:
                job_location.append(None)
                
            # Job level / Seniority (if any?)
            job_level_var = soup_level2.findAll("p", {"id": "seniority"})
            if len(job_level_var) > 0:
                job_level.append(job_level_var[0].text)
            else:
                job_level.append("Unknown")
                
            # Salary
            salary_var = soup_level2.findAll("span", "dib")
            if len(salary_var) > 0:
                if len(salary_var[1].text.split("to")) == 2:  # If there are 2 numbers given
                    minimum = int("".join(salary_var[1].text.split("to")[0].split("$")[1].split(",")))
                    maximum = int("".join(salary_var[1].text.split("to")[1].split("$")[1].split(",")))
                    min_salary.append(minimum)
                    max_salary.append(maximum)
                    salary_type.append(salary_var[-2].text)
                elif salary_var[1].text.split("to")[0] == "Salary undisclosed": # Special Case (Non Recruiter)
                    min_salary.append(None)
                    max_salary.append(None)
                    salary_type.append(None)
                elif salary_var[2].text.split("to")[0] == "Salary undisclosed": # Special Case (Recruiter)
                    min_salary.append(None)
                    max_salary.append(None)
                    salary_type.append(None)
                else:  # If there is only 1 number given
                    minimum = int("".join(salary_var[2].text.split("to")[0].split("$")[1].split(",")))
                    maximum = int("".join(salary_var[2].text.split("to")[1].split("$")[1].split(",")))
                    min_salary.append(minimum)
                    max_salary.append(maximum)
                    salary_type.append(salary_var[-2].text)
            else:  # If there are 0 number given
                min_salary.append(0)
                max_salary.append(0)
                
            # Job Type
            job_type_var = soup_level2.findAll('p', {"id":"employment_type"})
            list1 = ["Contract", "Internship", "Temporary", "Permanent"]
            list2 = [is_contract, is_intern, is_temp, is_perm]
            if len(job_type_var) > 0:
                job_type.append(job_type_var[0].text)
                for num, pos in enumerate(list1):
                    if pos in job_type_var[0].text.split(", "):
                        list2[num].append(1)
                    else:
                        list2[num].append(0)
            else:
                job_type.append("Unknown")
                for num, pos in enumerate(list1):
                    list2[num].append(0)
                    
            # Job Industry
            job_industry_var = soup_level2.findAll("p", {"id": "job-categories"})
            if len(job_industry_var) > 0:
                job_industry.append(job_industry_var[0].text)
            else:
                job_industry.append(None)
                
            # Job Description
            job_description_var = soup_level2.findAll("div", {"id":"description-content"})
            if job_description_var:
                job_desc.append(job_description_var[0].text)
            else:
                job_desc.append("None")
                
            # Job Requirements
            job_requirements_var = soup_level2.findAll("div", {"id":"requirements-content"})
            if job_requirements_var:
                job_req.append(job_requirements_var[0].text)
            else:
                job_req.append("None")
                
            # Days Posted
            days_posted_var = soup_level2.findAll("span", {"id":"last_posted_date"})
            if len(days_posted_var) > 0:
                text = days_posted_var[0].text.split("Posted ")[1]
                days_posted.append(text)
            else:
                days_posted.append("None")
                
            driver.back()   
            
            job_count+=1
            jip_count+=1
            if job_count == minreq:
                break
    else:
        if (soup.findAll("span", {"type":"action"})[-1].text == "❯"):
            button = driver.find_element_by_xpath('//*[@type="action"][last()]')
            button.click()
            sleep(timer)
            soup = BeautifulSoup(driver.page_source, 'lxml')
            jip_count = 0
            try:
                # Dismiss Add Skills Popup
                button = driver.find_element_by_xpath('//*[@class="joyride-tooltip__close"]')
                button.click()
            except:
                continue
        else:
            driver.close()
            print("Done")
            break

In [ ]:
# Convert to Pandas DataFrame
df = pd.DataFrame([job_id, job_url, job_title, job_type, company_name, job_location, job_level, job_industry, min_salary, \
                   max_salary, salary_type, days_posted, is_contract, is_intern, is_temp, is_perm, job_desc, \
                   job_req], index=["Job ID", "URL", "Job Title", "Job Type", "Company Name", "Job Location", "Job Level", \
                                    "Job Industry", "Min Salary", "Max Salary", "Salary Paid", "Date Posted", "Contract", \
                                    "Internship", "Temporary", "Permanent", "Job Description", "Job Requirements"]).T

# Drop any duplicate data row
df.drop_duplicates(keep=False,inplace=True) 

# Exporting dataframe to CSV file
df.to_csv(output+'.csv', encoding='utf-8')